In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import umap.umap_ as umap
import os

import warnings
from matplotlib import MatplotlibDeprecationWarning

warnings.filterwarnings("ignore", category=MatplotlibDeprecationWarning)
plt.style.use('seaborn-white')


/home/mehul/.conda/envs/pyg/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mehul/.conda/envs/pyg/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mehul/.conda/envs/pyg/lib/python3.11/site-packages/umap/distances.py:1086: Num

In [8]:
import os
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


def get_data(epoch, base_location, train=True):
    tsne_folder = "train_tsne" if train else "test_tsne"

    # Define the file paths using the base location
    x_file_paths = [
        os.path.join(base_location, tsne_folder, f"x_epoch_{epoch}_iter_{i}.npy") for i in range(6)
    ]

    label_file_paths = [
        os.path.join(base_location, tsne_folder, f"label_epoch_{epoch}_iter_{i}.npy") for i in range(6)
    ]

    # Load the x files
    x_arrays = [np.load(file_path) for file_path in x_file_paths]
    # Load the label files
    label_arrays = [np.load(file_path) for file_path in label_file_paths]

    # Concatenate arrays along the first axis
    x_final = np.concatenate(x_arrays, axis=0)
    label_final = np.concatenate(label_arrays, axis=0)

    return x_final, label_final


def plot_data(x_final, label_final, base_location, epoch, train=True):
    tsne = TSNE(angle=0.2, init='pca', method='barnes_hut',
                metric='euclidean', n_iter=2000, perplexity=50, random_state=42)
    x_tsne = tsne.fit_transform(x_final)

    # Plotting
    plt.figure(figsize=(7, 6), dpi=150)
    plt.style.use('seaborn-white')

    label_classes = [0, 1]
    colors = ['blue', 'red']
    labels = ["TD", "ASD"]

    for i, (c, label) in enumerate(zip(colors, labels)):
        plt.scatter(x_tsne[label_final == i, 0], x_tsne[label_final == i, 1], marker='o',
                    color=c, label=label, edgecolor='w', s=50, alpha=0.8)

    # Determine the title text based on the base location
    if base_location.endswith("14-48-41"):
        title_text = 'HyperGEL'
    elif base_location.endswith("14-48-22"):
        title_text = 'HyperGraphGCN'
    else:
        title_text = 'GCN'

    # Customize plot attributes
    plt.xticks([])
    plt.yticks([])
    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    leg = plt.legend(loc='best', fontsize=10, frameon=True, edgecolor='black')
    for lh in leg.legendHandles:
        lh.set_alpha(1)

    # Save the plot
    directory_name = os.path.basename('.')
    train_test_folder = "train" if train else "test"
    save_directory = os.path.join(
        '/home/mehul/asd_graph/Notebooks', f"{directory_name}_plots", train_test_folder, title_text)

    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    plt.axis('off')
    plt.savefig(f"{save_directory}/epoch_{epoch}.png",
                bbox_inches='tight', dpi=150)
    plt.close()

    return save_directory


def get_data_and_plot(epoch, base_location, train=True):
    x_final, label_final = get_data(epoch, base_location, train=train)
    save_directory = plot_data(
        x_final, label_final, base_location, epoch, train=train)
    return save_directory


In [9]:
base_locations = [
    "/home/mehul/asd_graph/baselines/outputs/2023-09-05/22-08-45",
    "/home/mehul/asd_graph/baselines/outputs/2023-09-05/22-09-04",
    "/home/mehul/asd_graph/baselines/outputs/2023-09-05/22-09-15"
]

for base_location in base_locations:
    for epoch in range(100):  # For epochs 0-99
        save_directory = get_data_and_plot(epoch, base_location, train=True)
        save_directory = get_data_and_plot(epoch, base_location, train=False)

    print(f"Saved epoch {epoch} to {save_directory}")


Saved epoch 99 to /home/mehul/asd_graph/Notebooks/._plots/test/GCN


KeyboardInterrupt: 

In [13]:
base_locations = [
    "/home/mehul/asd_graph/baselines/outputs/2023-09-05/14-48-41",  # HyperGEL
    "/home/mehul/asd_graph/baselines/outputs/2023-09-05/14-48-22",  # HypergraphGCN
    "/home/mehul/asd_graph/baselines/outputs/2023-09-05/14-54-55"   # GCN
]
print("HyperGEL model:")
get_data_and_plot(26, base_locations[0])

print("HypergraphGCN model:")
get_data_and_plot(4, base_locations[1])
get_data_and_plot(50, base_locations[1])


print("GCN model:")
get_data_and_plot(13, base_locations[2])
get_data_and_plot(29, base_locations[2])


HyperGEL model:


/tmp/ipykernel_3433292/1470260004.py:39: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')  # No grid, plain white background
/tmp/ipykernel_3433292/1470260004.py:70: MatplotlibDeprecationWarning: The legendHandles attribute was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use legend_handles instead.
  for lh in leg.legendHandles:


HypergraphGCN model:


/tmp/ipykernel_3433292/1470260004.py:39: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')  # No grid, plain white background
/tmp/ipykernel_3433292/1470260004.py:70: MatplotlibDeprecationWarning: The legendHandles attribute was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use legend_handles instead.
  for lh in leg.legendHandles:
/tmp/ipykernel_3433292/1470260004.py:39: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')  # No grid, plain white back

GCN model:


/tmp/ipykernel_3433292/1470260004.py:39: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')  # No grid, plain white background
/tmp/ipykernel_3433292/1470260004.py:70: MatplotlibDeprecationWarning: The legendHandles attribute was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use legend_handles instead.
  for lh in leg.legendHandles:
/tmp/ipykernel_3433292/1470260004.py:39: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')  # No grid, plain white back

'/home/mehul/asd_graph/Notebooks/._plots/GCN'

In [19]:
get_data_and_plot(52, base_locations[1])


/tmp/ipykernel_3433292/3111290558.py:39: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')  # No grid, plain white background


TypeError: scatter() got multiple values for argument 's'

<Figure size 1050x900 with 0 Axes>

In [ ]:

# for base_location in base_locations:
#     for epoch in range(100):  # For epochs 0-99
#         save_directory = get_data_and_plot(epoch, base_location)
#         print(f"Saved epoch {epoch} to {save_directory}")


In [ ]:

model2locations = {
    'HyperGraphGCN': "/home/mehul/asd_graph/baselines/outputs/2023-09-05/10-51-29",
    'GCN': "/home/mehul/asd_graph/baselines/outputs/2023-09-03/11-31-08",
    'HyperGEL': "/home/mehul/asd_graph/baselines/outputs/2023-09-05/10-37-58"
}
model2epochs = {
    'HyperGraphGCN': 37,
    'GCN': 7,
    'HyperGEL': 11
}
models = ['HyperGraphGCN', 'GCN', 'HyperGEL']
for model in models:
    base_location = model2locations[model]
    epoch = model2epochs[model]
    x_final, label_final = get_data(epoch, base_location)
    plot_data(x_final, label_final, base_location, epoch)
